In [239]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from xgboost import XGBRegressor
warnings.filterwarnings("ignore")

%matplotlib inline

In [240]:
df = pd.read_excel(r"./Data/Data.xlsx")
print(df.shape)
df.head()

(10683, 11)


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,Null,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,Null,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,Null,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,Null,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,Null,13302


In [241]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [242]:
df['Duration'].value_counts()

Duration
2h 50m     550
1h 30m     386
2h 55m     337
2h 45m     337
2h 35m     329
          ... 
30h 10m      1
31h 30m      1
42h 5m       1
4h 10m       1
5m           1
Name: count, Length: 368, dtype: int64

In [243]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [244]:
df.dropna(inplace=True)

In [245]:
df.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

In [246]:
df.duplicated().sum()

np.int64(220)

In [247]:
df.drop_duplicates(inplace=True)

In [248]:
df['Journey_day'] = pd.to_datetime(df['Date_of_Journey'], format="%d/%m/%Y").dt.day

In [249]:
df['Journey_month'] = pd.to_datetime(df['Date_of_Journey'], format="%d/%m/%Y").dt.month

In [250]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_day,Journey_month
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,Null,3897,24,3
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,Null,7662,1,5
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,Null,13882,9,6
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,Null,6218,12,5
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,Null,13302,1,3


In [251]:
df.drop(['Date_of_Journey'],axis=1, inplace=True)

In [252]:
df['Dep_hour'] = pd.to_datetime(df['Dep_Time']).dt.hour
df['Dep_min'] = pd.to_datetime(df['Dep_Time']).dt.minute

df.drop(['Dep_Time'],axis=1, inplace=True)

In [253]:
df.head()

,Airline,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_day,Journey_month,Dep_hour,Dep_min
0,IndiGo,Banglore,New Delhi,BLR → DEL,01:10 22 Mar,2h 50m,non-stop,Null,3897,24,3,22,20
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,13:15,7h 25m,2 stops,Null,7662,1,5,5,50
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,04:25 10 Jun,19h,2 stops,Null,13882,9,6,9,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,23:30,5h 25m,1 stop,Null,6218,12,5,18,5
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,21:35,4h 45m,1 stop,Null,13302,1,3,16,50


In [254]:
df['Arrival_hour'] = pd.to_datetime(df['Arrival_Time']).dt.hour
df['Arrival_min'] = pd.to_datetime(df['Arrival_Time']).dt.minute

df.drop(['Arrival_Time'],axis=1, inplace=True)

In [255]:
import re

# Function to extract hours and minutes
def extract_time(time_str):
    hours = re.search(r'(\d+)h', time_str)
    minutes = re.search(r'(\d+)m', time_str)
    
    hours = int(hours.group(1)) if hours else 0
    minutes = int(minutes.group(1)) if minutes else 0
    
    return {'hours': hours, 'minutes': minutes}

# Apply function to the DataFrame
df[["Duration_hours", "Duration_mins"]] = df['Duration'].apply(lambda x: pd.Series(extract_time(x)))

In [256]:
df.drop(['Duration'],axis=1, inplace=True)

In [257]:
df.drop(["Route", "Additional_Info"], axis = 1, inplace = True)

In [258]:
columns = df.columns.tolist()

In [259]:
columns.remove('Price')

In [260]:
columns.append('Price')

In [261]:
df = df[columns]

In [262]:
df = df =  df.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4})

In [263]:
x = df.drop(['Price'], axis=1)
y = df['Price']

In [264]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [265]:
x_train.head()

,Airline,Source,Destination,Total_Stops,Journey_day,Journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins
6727,IndiGo,Mumbai,Hyderabad,0,1,5,9,10,10,40,1,30
6322,Jet Airways,Delhi,Cochin,1,9,5,10,0,19,0,9,0
6874,Multiple carriers,Delhi,Cochin,1,1,6,10,0,1,30,15,30
3493,IndiGo,Kolkata,Banglore,1,6,5,8,10,13,0,4,50
611,Air India,Delhi,Cochin,1,21,3,23,0,19,15,20,15


In [266]:
x_train.iloc[:,1]

6727      Mumbai
6322       Delhi
6874       Delhi
3493     Kolkata
611        Delhi
          ...   
5799    Banglore
5247    Banglore
5452       Delhi
861        Delhi
7367     Chennai
Name: Source, Length: 8369, dtype: object

In [267]:
df.head()

,Airline,Source,Destination,Total_Stops,Journey_day,Journey_month,Dep_hour,Dep_min,Arrival_hour,Arrival_min,Duration_hours,Duration_mins,Price
0,IndiGo,Banglore,New Delhi,0,24,3,22,20,1,10,2,50,3897
1,Air India,Kolkata,Banglore,2,1,5,5,50,13,15,7,25,7662
2,Jet Airways,Delhi,Cochin,2,9,6,9,25,4,25,19,0,13882
3,IndiGo,Kolkata,Banglore,1,12,5,18,5,23,30,5,25,6218
4,IndiGo,Banglore,New Delhi,1,1,3,16,50,21,35,4,45,13302


In [268]:
trf1 = ColumnTransformer(
    transformers=[
        ('ohe_air', OneHotEncoder(drop='first', dtype='int'),[0]),
        ('ohe_source', OneHotEncoder(drop='first', dtype='int'), [1]),
        ('ohe_destination', OneHotEncoder(drop='first', dtype='int',), [2])
        
    ],
    remainder='passthrough'  # Keeps other columns as-is
)

In [269]:
from sklearn.preprocessing import StandardScaler
trf2 = ColumnTransformer([
    ('scale', StandardScaler(), slice(0,35))
])

In [270]:
trf3 = XGBRegressor()

In [271]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3)
])

In [272]:
pipe.fit(x_train,y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_air',
                                                  OneHotEncoder(drop='first',
                                                                dtype='int'),
                                                  [0]),
                                                 ('ohe_source',
                                                  OneHotEncoder(drop='first',
                                                                dtype='int'),
                                                  [1]),
                                                 ('ohe_destination',
                                                  OneHotEncoder(drop='first',
                                                                dtype='int'),
                                                  [2])])),
                ('trf2',
                 ColumnTransformer(transformers=[('scale', StandardScaler(),
                                                  slice(0, 35, None))])),
                (...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, random_state=None, ...))])

In [273]:
pipe.named_steps['trf1'].transformers_[1]

('ohe_source', OneHotEncoder(drop='first', dtype='int'), [1])

In [274]:
y_pred = pipe.predict(x_test)

In [275]:
for step_name, step_process in pipe.named_steps.items():
    print(f"{step_name}: {step_process}")

trf1: ColumnTransformer(remainder='passthrough',
                  transformers=[('ohe_air',
                                 OneHotEncoder(drop='first', dtype='int'),
                                 [0]),
                                ('ohe_source',
                                 OneHotEncoder(drop='first', dtype='int'),
                                 [1]),
                                ('ohe_destination',
                                 OneHotEncoder(drop='first', dtype='int'),
                                 [2])])
trf2: ColumnTransformer(transformers=[('scale', StandardScaler(),
                                 slice(0, 35, None))])
trf3: XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,

In [276]:
print(x_train.isna().sum())
print(y_train.isna().sum())

Airline           0
Source            0
Destination       0
Total_Stops       0
Journey_day       0
Journey_month     0
Dep_hour          0
Dep_min           0
Arrival_hour      0
Arrival_min       0
Duration_hours    0
Duration_mins     0
dtype: int64
0


In [281]:
from sklearn.model_selection import cross_val_score
training_accuracy = cross_val_score(pipe, x_train, y_train, cv=5)

In [282]:
print('training_accuracy for all 10 indivisual :', training_accuracy)
print()
print("training_accuracy with mean value :", training_accuracy.mean())
print()
print("training_accuracy max value :", training_accuracy.max())

training_accuracy for all 10 indivisual : [       nan 0.81615257 0.87285322 0.82678509 0.803985  ]

training_accuracy with mean value : nan

training_accuracy max value : nan


In [194]:
df.isna().sum()

Airline           0
Source            0
Destination       0
Total_Stops       0
Journey_day       0
Journey_month     0
Dep_hour          0
Dep_min           0
Arrival_hour      0
Arrival_min       0
Duration_hours    0
Duration_mins     0
Price             0
dtype: int64